# Prepare Data for Shiny

In this notebook we prepare the data so that it can be better visualized in an R Shiny application.


#### Before you begin

When you import this project on an IBM Cloud environment, insert a project access token at the top of this notebook as a code cell. <br>
If you do not see the cell above, Insert a project token: Click on **More -> Insert project token** in the top-right menu section and run the cell. <br>
![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

## Step 1: Read and Preview the Original File

In [2]:
!ls -lh

total 49M
-rw-rw---- 1 wsuser wscommon 49M Nov  9 20:53 Historical_Product_Demand.csv


In [3]:
import pandas as pd

df = pd.read_csv('Historical_Product_Demand.csv')

In [4]:
df['Order_Demand'] = df['Order_Demand'].str.strip(' ')
df['Order_Demand'] = df['Order_Demand'].str.replace('(', '',regex=True)
df['Order_Demand'] = df['Order_Demand'].str.replace(')', '',regex=True)

df['Order_Demand'] = df['Order_Demand'].astype(int)
df.groupby('Warehouse')['Order_Demand'].sum()

Warehouse
Whse_A     159036821
Whse_C     585071404
Whse_J    3363200396
Whse_S    1038024700
Name: Order_Demand, dtype: int64

## Step 2: Prepare Data

### Add Location Information to the Warehouses

In [5]:
def set_wh(x):
    if x == 'Whse_J':
        return 'Los Angeles'
    elif x == 'Whse_A':
        return 'Eugene'
    elif x == 'Whse_S':
        return 'Seattle'
    else:
        return 'Sacramento'

df['Warehouse'] = df['Warehouse'].map(lambda x: set_wh(x))
df['Warehouse'].value_counts()

Los Angeles    764447
Eugene         153574
Seattle         88200
Sacramento      42354
Name: Warehouse, dtype: int64

### Generate Inventory Data

In [6]:
import random
df['Inventory'] = df['Order_Demand'].map(lambda x: random.randint(int(0.8*x), int(1.2*x)))

In [7]:
%time
df.dropna(inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Date_Month'] = pd.to_datetime(df['Date'].map(lambda x: str(int(x.year)) + '-' + str(int(x.month)) + '-' + '01'))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


### Add Details on State and Region

In [8]:
df_shiny = df.groupby(['Warehouse', 'Product_Code', 'Product_Category', 'Date_Month'])[['Order_Demand', 'Inventory']].sum().reset_index()
def set_state(x):
    if x == 'Los Angeles' or x == 'Sacramento':
        return 'CA'
    elif x == 'Seattle':
        return 'WA'
    else:
        return 'OR'
df_shiny['Warehouse_State'] = df_shiny['Warehouse'].map(lambda x: set_state(x))
df_shiny['Warehouse_Region'] = 'West Coast Region'
df_shiny.head()

Warehouse  Product_Code Product_Category Date_Month  Order_Demand  \
0    Eugene  Product_0001     Category_005 2012-01-01          6400   
1    Eugene  Product_0001     Category_005 2012-02-01         10400   
2    Eugene  Product_0001     Category_005 2012-03-01         11000   
3    Eugene  Product_0001     Category_005 2012-04-01          1500   
4    Eugene  Product_0001     Category_005 2012-05-01          8000   

   Inventory Warehouse_State   Warehouse_Region  
0       5684              OR  West Coast Region  
1      11310              OR  West Coast Region  
2      10309              OR  West Coast Region  
3       1619              OR  West Coast Region  
4       8957              OR  West Coast Region

### Calculate Percent Demand and Create a Binary Label to Indicate if Current Inventory would be Enough to Meet the Demand or Not

In [9]:
df_shiny['pct_demand'] = df_shiny['Inventory'] / df_shiny['Order_Demand']
df_shiny['if_enough'] = (df_shiny['pct_demand'] >= 1).astype(int)

df_shiny.head()

Warehouse  Product_Code Product_Category Date_Month  Order_Demand  \
0    Eugene  Product_0001     Category_005 2012-01-01          6400   
1    Eugene  Product_0001     Category_005 2012-02-01         10400   
2    Eugene  Product_0001     Category_005 2012-03-01         11000   
3    Eugene  Product_0001     Category_005 2012-04-01          1500   
4    Eugene  Product_0001     Category_005 2012-05-01          8000   

   Inventory Warehouse_State   Warehouse_Region  pct_demand  if_enough  
0       5684              OR  West Coast Region    0.888125          0  
1      11310              OR  West Coast Region    1.087500          1  
2      10309              OR  West Coast Region    0.937182          0  
3       1619              OR  West Coast Region    1.079333          1  
4       8957              OR  West Coast Region    1.119625          1

## Step 3: Save the Final File Back to the Project

In [ ]:
project.save_data('Monthly_Demand_Inventory.csv', df_shiny.to_csv(index=False), overwrite=True)


**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020 <br>
All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**